In [1]:
import os
import numpy as np
import panda as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
#from keras.applications.vgg16 import VGG16
#from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

ModuleNotFoundError: No module named 'panda'

In [2]:
# Setting. Can change.
CLASSES = ['00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40','41']

batch_size_init = 32
batch_size_finetuning = 16

lr_init = 1e-4
lr_finetuning = 1e-6

epochs_init = 10
epochs_finetuning = 30
epochs_all = epochs_init + epochs_finetuning

#input image width, height and shape
IMG_HEIGHT = 200
IMG_WIDTH = 200
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)


#define path
dataset_dir = 'data/dataset'
trained_model_dir = 'trained_model'
results_dir = 'results'

dir_name = 'VGG'

model_checkpoint_dir = f'{trained_model_dir}/{dir_name}'
if not os.path.exists(model_checkpoint_dir):
    os.makedirs(model_checkpoint_dir)
    
model_init = os.path.join(model_checkpoint_dir, 'model_pre_final.h5')
    
chart_stats_dir = f'{results_dir}/{dir_name}'
if not os.path.exists(chart_stats_dir):
    os.makedirs(chart_stats_dir)

train_dir = f'{dataset_dir}/small_data/train/'
test_dir = f'{dataset_dir}/test/'



In [3]:
def get_generator(batch_size, is_augment=False):
    if is_augment:
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            brightness_range=(0.4,0.9),
            horizontal_flip=True,
            validation_split=0.2
        )
    else:
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_input,
            validation_split=0.2
        )

    
    train_generator = datagen.flow_from_directory(
        train_dir, 
        subset='training',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=batch_size,
        classes=CLASSES,
        class_mode="categorical",
        shuffle=False
    )

    val_generator = datagen.flow_from_directory(
        train_dir,
        subset='validation',
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=batch_size,
        classes=CLASSES,
        class_mode="categorical",
        shuffle=False
    )
    
    return train_generator, val_generator

In [4]:
def get_weight(model_path):
    loaded_model = load_model(model_path)
    return loaded_model.get_weights()


def create_model(lr, weight=None):
    base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_shape=IMG_SHAPE)
    
    if not weight:
        for layer in base_model.layers:
            layer.trainable = False
            
    x = base_model.layers[-1].output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(rate=0.3)(x)
    predictions = Dense(len(CLASSES), activation='softmax')(x)
    model = Model(inputs=base_model.inputs, outputs=predictions)
    
    if weight:
        model.set_weights(weight)

    # compile model
    opt = Adam(lr=lr)
    model.compile(
        optimizer=opt,
        loss=categorical_crossentropy,
        metrics=['accuracy']
    )
    
    return model

In [5]:
train_generator, validation_generator = get_generator(batch_size_init)

Found 3360 images belonging to 42 classes.
Found 840 images belonging to 42 classes.


In [6]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = create_model(lr_init)
model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [7]:
hist_init = model.fit(
    x=train_generator,
    epochs=epochs_init,
    validation_data=validation_generator
)
model.save(model_init)

Train for 105 steps, validate for 27 steps
Epoch 1/10
105/105 [==============================] - 102s 971ms/step - loss: 4.0462 - accuracy: 0.0131 - val_loss: 3.9121 - val_accuracy: 0.0250
Epoch 2/10
105/105 [==============================] - 90s 856ms/step - loss: 3.8944 - accuracy: 0.0176 - val_loss: 3.8473 - val_accuracy: 0.0238
Epoch 3/10
105/105 [==============================] - 90s 853ms/step - loss: 3.8689 - accuracy: 0.0143 - val_loss: 3.8251 - val_accuracy: 0.0238
Epoch 4/10
105/105 [==============================] - 89s 852ms/step - loss: 3.8735 - accuracy: 0.0140 - val_loss: 3.8145 - val_accuracy: 0.0238
Epoch 5/10
105/105 [==============================] - 90s 856ms/step - loss: 3.8697 - accuracy: 0.0170 - val_loss: 3.8032 - val_accuracy: 0.0286
Epoch 6/10
105/105 [==============================] - 89s 851ms/step - loss: 3.8607 - accuracy: 0.0205 - val_loss: 3.7882 - val_accuracy: 0.0286
Epoch 7/10
105/105 [==============================] - 90s 853ms/step - loss: 3.8597 - 

In [8]:
weight = get_weight(model_init)

model = create_model(lr_finetuning, weight)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 99, 99, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 99, 99, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 99, 99, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [9]:
checkpoint = ModelCheckpoint(
        filepath=model_checkpoint_dir+'/model_weights_{epoch}.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
        save_freq='epoch'
)

train_generator, validation_generator = get_generator(batch_size_finetuning, True)

hist_fine = model.fit(
    x=train_generator,
    epochs=epochs_finetuning,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

Found 3360 images belonging to 42 classes.
Found 840 images belonging to 42 classes.
Train for 210 steps, validate for 53 steps
Epoch 1/30
209/210 [============================>.] - ETA: 1s - loss: 3.7887 - accuracy: 0.0275
Epoch 00001: val_loss improved from inf to 3.71586, saving model to trained_model/VGG/model_weights_1.h5
210/210 [==============================] - 331s 2s/step - loss: 3.7890 - accuracy: 0.0274 - val_loss: 3.7159 - val_accuracy: 0.0369
Epoch 2/30
209/210 [============================>.] - ETA: 1s - loss: 3.7878 - accuracy: 0.0245
Epoch 00002: val_loss improved from 3.71586 to 3.70898, saving model to trained_model/VGG/model_weights_2.h5
210/210 [==============================] - 321s 2s/step - loss: 3.7873 - accuracy: 0.0244 - val_loss: 3.7090 - val_accuracy: 0.0429
Epoch 3/30
209/210 [============================>.] - ETA: 1s - loss: 3.7912 - accuracy: 0.0263
Epoch 00003: val_loss did not improve from 3.70898
210/210 [==============================] - 320s 2s/step

KeyboardInterrupt: 

In [ ]:
def show_acc_loss_graph(hist_init, hist):
    acc = hist_init.history['accuracy'] + hist.history['accuracy']
    val_acc = hist_init.history['val_accuracy'] + hist.history['val_accuracy']
    loss = hist_init.history['loss'] + hist.history['loss']
    val_loss = hist_init.history['val_loss'] + hist.history['val_loss']
    
    # save accuracy graph image
    plt.plot(range(1, epochs_all+1), acc, marker='.', label='accuracy')
    plt.plot(range(1, epochs_all+1), val_acc, marker='.', label='val_accuracy')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.ylim(top=1.0) 
    plt.savefig(os.path.join(chart_stats_dir, 'accuracy.png'))
    plt.show()

    # save loss graph image
    plt.plot(range(1, epochs_all+1), loss, marker='.', label='loss')
    plt.plot(range(1, epochs_all+1), val_loss, marker='.', label='val_loss')
    plt.legend(loc='best')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.ylim(bottom=0.0) 
    plt.savefig(os.path.join(chart_stats_dir, 'loss.png'))
    plt.show()

show_acc_loss_graph(hist_init, hist_fine)


#### Run on Test Set to get submission csv file

In [2]:
#load the trained model back
trained_model = create_model(lr, model_weight_path)

NameError: name 'create_model' is not defined

In [ ]:
result_df = pd.DataFrame(columns=['filename', 'category'])
result_df

In [ ]:
#get the list of images for prediction
img_files = [file for file in os.listdir(test_dir)]

#preprocess the images and get predictions
for img_file in img_files:
    input_img = load_img(f'{test_dir}/{img_file}', target_size=(IMG_WIDTH, IMG_HEIGHT))
    img = img_to_array(input_img)
    img = img.reshape(1,IMG_WIDTH,IMG_HEIGHT,3)
    img = preprocess_input(img)
    prediction = trained_model.predict(img)
    
    predicted_class = CLASSES[prediction.argmax()]
    
    result_df = result_df.append({'filename':img_file, 'category': predicted_class}, ignore_index=True)

In [ ]:
result_df

In [ ]:
result_df.to_csv(chart_stats_dir+'/submission.csv', index=False)

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_model(lr_init)
model.summary()

In [ ]:
hist_init = model.fit(
    x=train_generator,
    epochs=epochs_init,
    validation_data=test_generator
)
model.save(model_init)

In [ ]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

In [ ]:
tf.test.is_gpu_available()


In [ ]:
tf.test.gpu_device_name() 

In [ ]:
tf.__version__

In [ ]:
conda install tensorflow-gpu=1.13

In [ ]:
VGG16_MODEL.trainable=False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(CLASSES),activation='softmax')




In [ ]:
model = tf.keras.Sequential([
  VGG16_MODEL,
  global_average_layer,
  prediction_layer
])

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(), 
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=["accuracy"])

In [ ]:
model.summary()
